#### Setup

- Initialize Spark session connecting to the Postgres DB

In [ ]:
import findspark
findspark.init()
import pyspark # only run after findspark.init()

from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession \
    .builder \
    .appName("MusicBrainz PostgreSQL Connection") \
    .config("spark.jars", "/Users/d.veragillard/edu/semester/WIM-1/big-data-advanced-database/bd-project/postgresql-42.7.1.jar") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

database_url = "jdbc:postgresql://localhost:5432/musicbrainz"
properties = {"user": "musicbrainz", "password": "musicbrainz", "driver": "org.postgresql.Driver"}

#### Data Collection

- Get the relevant data from Postgres
- Already do cleaning in this stage by only selecting relevant columns

First get general Artist and Area(The origin to predict):

In [ ]:
# Read data from artist and area tables
artist_df = spark.read.jdbc(url=database_url, table="artist", properties=properties).select("id", "name", "area")
area_df = spark.read.jdbc(url=database_url, table="area", properties=properties).select("id", "name", "type")

# Join artist and area tables for detailed country information
artist_country_df = artist_df.join(area_df, artist_df.area == area_df.id)

# Select relevant columns
artist_country_df = artist_country_df.select(artist_df.name, area_df.name.alias("country"))

# Show a sample of data
# artist_country_df.show()

Now add additional Artist/Country information that could hint about the artist country:

In [ ]:
# Reading additional tables that could have useful information
alias_df = spark.read.jdbc(url=database_url, table="artist_alias", properties=properties).select("artist", "name")
artist_credit_df = spark.read.jdbc(url=database_url, table="artist_credit", properties=properties).select("id", "name")
language_df = spark.read.jdbc(url=database_url, table="language", properties=properties).select("id", "name")
script_df = spark.read.jdbc(url=database_url, table="script", properties=properties).select("id", "name")

# Joining tables

# Joining artist with alias
# Don't use "name" as join column, because it's ambiguous. Use "alias"
# Same for the other joins
artist_alias_df = artist_df.join(alias_df, artist_df.id == alias_df.artist).select(artist_df.name, alias_df.name.alias("alias"))
artist_credit_joined_df = artist_df.join(artist_credit_df, artist_df.id == artist_credit_df.id).select(artist_df.name, artist_credit_df.name.alias("credit"))
artist_language_df = artist_df.join(language_df, artist_df.id == language_df.id).select(artist_df.name, language_df.name.alias("language"))
artist_script_df = artist_df.join(script_df, artist_df.id == script_df.id).select(artist_df.name, script_df.name.alias("script"))

# Combining all data into one dataframe
combined_artist_df = artist_country_df \
    .join(artist_alias_df, ["name"], "left_outer") \
    .join(artist_credit_joined_df, ["name"], "left_outer") \
    .join(artist_language_df, ["name"], "left_outer") \
    .join(artist_script_df, ["name"], "left_outer")

# Show a sample of data
# combined_artist_df.show()

#### Data preprocessing

##### Data cleaning

Handle missing data. F.ex all the NULLs:

In [ ]:
from pyspark.sql.functions import col, when


# TODO - Replacing with empty values causes errors with OneHotEncoder
# Replace NULL in all features as it could have an empty value with
# combined_artist_df = combined_artist_df.withColumn("alias", when(col("alias").isNull(), "").otherwise(col("alias")))
# combined_artist_df = combined_artist_df.withColumn("credit", when(col("credit").isNull(), "").otherwise(col("credit")))
# combined_artist_df = combined_artist_df.withColumn("script", when(col("script").isNull(), "").otherwise(col("script")))
# combined_artist_df = combined_artist_df.withColumn("language", when(col("language").isNull(), "").otherwise(col("language")))
# combined_artist_df = combined_artist_df.withColumn("country", when(col("country").isNull(), "").otherwise(col("country")))
# combined_artist_df = combined_artist_df.withColumn("name", when(col("name").isNull(), "").otherwise(col("name")))

# Replacing Null values with None as an empty string errors in OneHotEncoder
combined_artist_df = combined_artist_df.withColumn("alias", when(col("alias").isNull(), None).otherwise(col("alias")))
combined_artist_df = combined_artist_df.withColumn("credit", when(col("credit").isNull(), None).otherwise(col("credit")))
combined_artist_df = combined_artist_df.withColumn("script", when(col("script").isNull(), None).otherwise(col("script")))
combined_artist_df = combined_artist_df.withColumn("language", when(col("language").isNull(), None).otherwise(col("language")))
combined_artist_df = combined_artist_df.withColumn("country", when(col("country").isNull(), None).otherwise(col("country")))
combined_artist_df = combined_artist_df.withColumn("name", when(col("name").isNull(), None).otherwise(col("name")))

# Dropping rows where 'country' or 'name' is null or empty
combined_artist_df = combined_artist_df.filter(combined_artist_df.country.isNotNull())
combined_artist_df = combined_artist_df.filter(combined_artist_df.name.isNotNull())

# Show a sample of data
# combined_artist_df.show()


##### Feature transformation

Transform feature strings into more suitable formats. To do this:

1. Use `StringIndexer` to convert the strings in the columns into indices(Like unique IDs)
2. Then use `OneHotEncoder` to convert the categorical indices into a binary vector(F.ex `[0,1,0,...]`)


In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
from pyspark.ml import Pipeline

# String Indexing for all categorical columns
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(combined_artist_df) 
            for column in ["name", "country", "alias", "credit", "language", "script"]]

# One-Hot Encoding for all indexed columns
encoders = [OneHotEncoder(inputCol=indexer.getOutputCol(), outputCol=indexer.getOutputCol()+"_vec") 
            for indexer in indexers]

##### Feature normalization

Scale transformed values to fixed range. To do this:

1. Use `VectorAssembler` to combine multiple columns into a single vector column. Helps with machine learning algorithms
2. Then apply `StandardScaler`. It helps, to make sure that the model is not influenced by features with larger scales

In [ ]:
# Vector Assembling all the features
assemblerInputs = [encoder.getOutputCol() for encoder in encoders]
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")

# Feature normalization
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")

Finally combine all steps into one transformation / cleaning pipeline and run it:

In [ ]:
# Building a Pipeline for transformations
pipeline = Pipeline(stages=indexers + encoders + [assembler, scaler])

# TODO - Empty sting values (in alias) cause errors with OneHotEncoder
# Print which column is indexed
# print("StringIndexer Input Column: ", indexers[2].getInputCol(), "Output Column: ", indexers[2].getOutputCol())
# pipeline = Pipeline(stages=[indexers[2], encoders[2]])

# Transforming the data
model = pipeline.fit(combined_artist_df)
transformed_df = model.transform(combined_artist_df)

# Showing the processed DataFrame
transformed_df.select("scaledFeatures").show()

#### Data Splitting

- Generate test, train and validation datasets


#### Training

- Select model
- Train model

#### Evaluation

- Validate model performance
- Adjust parameters respectively (Hyperparameter Tuning, ...)

#### Testing

- On unseen Data

#### Deployment

- Deploy into Cloud?

#### Monitoring / Maintainance

- ?